Kattson Bastos

22, May, 2022

## 0. IMPORTING

In [12]:
from fastparquet import ParquetFile


from sklearn.metrics             import mean_squared_error

from sklearn.linear_model       import LinearRegression
from sklearn.linear_model       import Lasso
from sklearn.linear_model       import Ridge

from sklearn.feature_extraction import DictVectorizer

### 0.2. Constant Variables

In [13]:
BASE_PATH = '../data/'

## 1. DATA LOADING AND DESCRIPTION

In [14]:
data_jan = ParquetFile(BASE_PATH + 'yellow_2022_01.parquet').to_pandas()
data_feb = ParquetFile(BASE_PATH + 'yellow_2022_02.parquet').to_pandas()

### 1.1. Data Dimensions

In [17]:
print('Number of Rows (Jan): ', data_jan.shape[0])
print('Number of Columns (Jan): ', data_jan.shape[1])

Number of Rows (Jan):  2463931
Number of Columns (Jan):  19


In [18]:
print('Number of Rows (Feb): ', data_feb.shape[0])
print('Number of Columns (JanuFebary): ', data_feb.shape[1])

Number of Rows (Feb):  2979431
Number of Columns (JanuFebary):  19


### 1.2. Missing Values

Let's check if there are missing value in the dataset.

In [19]:
data_jan.isna().sum()

VendorID                     0
tpep_pickup_datetime         0
tpep_dropoff_datetime        0
passenger_count          71503
trip_distance                0
RatecodeID               71503
store_and_fwd_flag       71503
PULocationID                 0
DOLocationID                 0
payment_type                 0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
congestion_surcharge     71503
airport_fee              71503
dtype: int64

In [20]:
data_feb.isna().sum()

VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          101738
trip_distance                 0
RatecodeID               101738
store_and_fwd_flag       101738
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     101738
airport_fee              101738
dtype: int64

## ANSWERING HOMEWORK QUESTIONS

### Q1: How many records are there for Jan?

In [21]:
print('Number of Rows (Jan): ', data_jan.shape[0])

Number of Rows (Jan):  2463931


### Q2: What's the average trip duration in January?

In [23]:
data_jan['duration'] = data_jan['tpep_dropoff_datetime'] - data_jan['tpep_pickup_datetime']

In [24]:
data_jan['duration'] = data_jan.duration.apply(lambda td: td.total_seconds() / 60)

In [25]:
print(f"The std trip duration in January is {round(data_jan['duration'].std(), 3)} minutes")

The std trip duration in January is 46.445 minutes


### Removing Outliers

In [26]:
data_jan = data_jan[(data_jan.duration >= 1) & (data_jan.duration <= 60)]

In [27]:
data_jan.shape

(2421440, 20)

In [28]:
data_jan.shape[0] / 2463931

0.9827547930522406

### Q3: One-hot encoding

In [30]:
data_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [31]:
categorical = ['PULocationID', 'DOLocationID']
data_jan[categorical] = data_jan[categorical].astype(str)

/tmp/ipykernel_4782/29949242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_jan[categorical] = data_jan[categorical].astype(str)


In [33]:
train_dicts = data_jan[categorical].to_dict(orient='records')

dv = DictVectorizer()

In [34]:
X_train = dv.fit_transform(train_dicts)

X_train.shape

(2421440, 515)

In [35]:
print(f"The number of columns of the matrix is {X_train.shape[1]}.")

The number of columns of the matrix is 515.


### Q5: What's the RMSE on train?

In [36]:
target = 'duration'
y_train = data_jan[target].values

#### Fitting the model

In [37]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [38]:
print(f"The RMSE on train id {mean_squared_error(y_train, y_pred, squared=False)}")

The RMSE on train id 6.986190841034851


### Q6: What's the RMSE on validation?

### Preparing the february data

In [42]:
#duration
data_feb['duration'] = data_feb['tpep_dropoff_datetime'] - data_feb['tpep_pickup_datetime']
data_feb['duration'] = data_feb.duration.apply(lambda td: td.total_seconds() / 60)

# removing outliers
data_feb = data_feb[(data_feb.duration >= 1) & (data_feb.duration <= 60)]

# filling missing values
#data_feb['PULocationID'] = data_feb['PULocationID'].fillna(-1)
#data_feb['DOLocationID'] = data_feb['DOLocationID'].fillna(-1)

# one-hot encoding
categorical = ['PULocationID', 'DOLocationID']
data_feb[categorical] = data_feb[categorical].astype(str)

test_dicts = data_feb[categorical].to_dict(orient='records')

X_val = dv.transform(test_dicts)

# target varaible
y_val = data_feb[target].values

In [43]:
y_pred = lr.predict(X_val)

In [44]:
print(f"The RMSE on validation id {mean_squared_error(y_val, y_pred, squared=False)}")

The RMSE on validation id 7.786408695263233
